In [1]:
from pathlib import Path
import json
import pandas as pd

try:
    from IPython.display import display  # type: ignore
except Exception:
    # Fallback for non-notebook execution
    display = print

# Reload local notebook utilities to pick up edits without restarting kernel
import importlib
import testLibs as tl
importlib.reload(tl)

# Notebook helpers (scoring + flatteners)
ResultsFlattener = tl.ResultsFlattener
mznResultsFlattener = tl.mznResultsFlattener
get_significative_solvers = tl.get_significative_solvers
scoreComputation_subset = tl.scoreComputation_subset
compute_llm_scores = tl.compute_llm_scores
compute_top1_llm_scores = tl.compute_top1_llm_scores
compute_closed_gap = tl.compute_closed_gap
build_llm_performance_table = tl.build_llm_performance_table
filter_to_solvers = tl.filter_to_solvers
singleSolverScore = tl.singleSolverScore

# Optional plotting libs (only needed for plots)
try:
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    plt = None
try:
    import seaborn as sns
except ModuleNotFoundError:
    sns = None

In [2]:
data_dir = Path("../data/testOutputSignificative").resolve()
if not data_dir.is_dir():
    raise FileNotFoundError(f"Expected folder not found: {data_dir}")

json_paths = sorted(data_dir.glob("*.json"))
print(f"Loading {len(json_paths)} JSON files from {data_dir}")

data_by_file = {}
for p in json_paths:
    with p.open("r", encoding="utf-8") as f:
        data_by_file[p.name] = json.load(f)

# List in a stable order, matching json_paths
data_list = [data_by_file[p.name] for p in json_paths]

print("Loaded files:")
for name in data_by_file.keys():
    print(" -", name)

with open('../data/tablesJSON/allTables_free.json', 'r') as f1:
    MznResults = json.load(f1)

Loading 4 JSON files from /home/vro5/Coding/AgenticSolvers/test/data/testOutputSignificative
Loaded files:
 - LLMsuggestions_significative_uncommented_fzn.json
 - LLMsuggestions_significative_uncommented_fzn_solverdesc.json
 - LLMsuggestions_significative_uncommented_fzncat.json
 - LLMsuggestions_significative_uncommented_fzncat_solverdesc.json


In [3]:
# --- Significative-only scoring pipeline ---
sig_solvers = get_significative_solvers()
print(f"Using significative solvers (count={len(sig_solvers)}):")
for s in sig_solvers:
    print(" -", s)

# MiniZinc results -> scores restricted to significative solvers
mzn_raw_df = mznResultsFlattener(MznResults)
scored_sig_df = scoreComputation_subset(mzn_raw_df, allowed_solvers=sig_solvers)
print(f"\nMZN rows (raw): {len(mzn_raw_df)}")
print(f"MZN rows (significative-only): {len(scored_sig_df)}")

# Optional: show best single solver within significative set
sig_single_solver_rank = singleSolverScore(scored_sig_df)
display(sig_single_solver_rank.head(10))

# LLM results -> compute Top-3, Top-1, ClosedGap within significative set
tables_by_file = {}
for fname, llm_results in data_by_file.items():
    llm_df = ResultsFlattener(llm_results)
    top3_summary = compute_llm_scores(llm_df, scored_sig_df, allowed_solvers=sig_solvers)
    top1_summary, top1_scored = compute_top1_llm_scores(llm_df, scored_sig_df, allowed_solvers=sig_solvers)
    cg_rows = compute_closed_gap(top1_scored, scored_sig_df, allowed_solvers=sig_solvers, sbs_solver=None)
    cg_df = pd.DataFrame(cg_rows)
    perf_table = build_llm_performance_table(
        top3_summary=top3_summary,
        top1_summary=top1_summary,
        closed_gap=cg_df,
        sort_by='SingleScore',
        ascending=False,
    )
    tables_by_file[fname] = {
        "top3_summary": top3_summary,
        "top1_summary": top1_summary,
        "closed_gap": cg_df,
        "performance_table": perf_table,
    }

print(f"\nComputed significative-only scoring for {len(tables_by_file)} LLM result files.")

# Display intermediate tables + performance table for each loaded file (stable order)
for p in json_paths:
    fname = p.name
    if fname not in tables_by_file:
        continue
    print("\n" + "=" * 80)
    print("File:", fname)
    print("=" * 80)

    print("\nTop-3 summary:")
    display(tables_by_file[fname]["top3_summary"])

    print("\nTop-1 summary:")
    display(tables_by_file[fname]["top1_summary"])

    print("\nClosed-gap table:")
    display(tables_by_file[fname]["closed_gap"])

    print("\nPerformance table:")
    display(tables_by_file[fname]["performance_table"])

Using significative solvers (count=12):
 - cbc-free
 - choco-solver__cp_-free
 - choco-solver__cp-sat_-free
 - cp_optimizer-free
 - cplex-free
 - gurobi-free
 - highs-free
 - izplus-free
 - jacop-free
 - pumpkin-free
 - scip-free
 - sicstus_prolog-free

MZN rows (raw): 2000
MZN rows (significative-only): 1200


,Solver,TotalScore,NumOptimal
5,gurobi-free,55.449377,38
9,pumpkin-free,57.232777,33
1,choco-solver__cp-sat_-free,60.303673,32
4,cplex-free,48.631992,30
2,choco-solver__cp_-free,58.330593,29
3,cp_optimizer-free,51.075820,26
8,jacop-free,43.847220,25
7,izplus-free,58.000135,25
11,sicstus_prolog-free,43.918052,24
10,scip-free,36.490140,20



Computed significative-only scoring for 4 LLM result files.

File: LLMsuggestions_significative_uncommented_fzn.json

Top-3 summary:


,provider,model,LLM_TotalScore,InstancesCovered,LLM_AvgScore
0,groq,openai/gpt-oss-120b,72.347686,100,0.723477



Top-1 summary:


...

,provider,model,LLM_Top1_TotalScore,LLM_Top1_AvgScore,InstancesCovered
0,groq,openai/gpt-oss-120b,51.07582,0.53764,100



Closed-gap table:


,provider,model,InstancesCovered,AS,SBS,VBS,ClosedGap
0,groq,openai/gpt-oss-120b,100,51.07582,60.303673,83.75,-0.393573



Performance table:


,Model,Single Score,Parallel Score,Closed Gap
0,openai/gpt-oss-120b,51.07582,66.854373,-0.393573


# Temperature sampling analysis (significative solvers)

This section loads LLM suggestion outputs generated with different decoding temperatures from `../data/testOutputSigTemp/` and evaluates them with the same scoring pipeline (Top-3, Top-1, ClosedGap) restricted to the significative solver set.

In [4]:
# --- Temperature sampling analysis ---
import re

temp_data_dir = Path("../data/testOutputSigTemp").resolve()
if not temp_data_dir.is_dir():
    raise FileNotFoundError(f"Expected folder not found: {temp_data_dir}")

temp_json_paths = sorted(temp_data_dir.glob("*.json"))
print(f"Loading {len(temp_json_paths)} temperature-sampling JSON files from {temp_data_dir}")

def _parse_temperature_from_filename(name: str) -> float | None:
    # Expected suffix patterns: _T0.json, _T0p2.json, ...
    m = re.search(r"_T(?P<t>\d+(?:p\d+)?)\.json$", name)
    if not m:
        return None
    t_str = m.group("t").replace("p", ".")
    try:
        return float(t_str)
    except ValueError:
        return None

def _parse_variant_from_filename(name: str) -> str:
    # Extract between 'LLMsuggestions_significative_' and '_T...'
    m = re.search(r"^LLMsuggestions_significative_(?P<v>.+)_T\d+(?:p\d+)?\.json$", name)
    return m.group("v") if m else "unknown"

# Ensure baseline scoring tables exist (cell 3 computes these)
if "scored_sig_df" not in globals():
    raise RuntimeError("Run the significative-only scoring pipeline cell first to create `scored_sig_df`.")
if "sig_solvers" not in globals():
    sig_solvers = get_significative_solvers()

temp_tables_by_file = {}
for p in temp_json_paths:
    fname = p.name
    with p.open("r", encoding="utf-8") as f:
        llm_results_wrapped = json.load(f)

    # Files are structured like {'chat': {...}}; we evaluate the entire object as-is.
    llm_df = ResultsFlattener(llm_results_wrapped)
    top3_summary = compute_llm_scores(llm_df, scored_sig_df, allowed_solvers=sig_solvers)
    top1_summary, top1_scored = compute_top1_llm_scores(llm_df, scored_sig_df, allowed_solvers=sig_solvers)
    cg_rows = compute_closed_gap(top1_scored, scored_sig_df, allowed_solvers=sig_solvers, sbs_solver=None)
    cg_df = pd.DataFrame(cg_rows)
    perf_table = build_llm_performance_table(
    top3_summary=top3_summary,
    top1_summary=top1_summary,
    closed_gap=cg_df,
    sort_by='SingleScore',
    ascending=False,
    )
    temp_tables_by_file[fname] = {
        "temperature": _parse_temperature_from_filename(fname),
        "variant": _parse_variant_from_filename(fname),
        "top3_summary": top3_summary,
        "top1_summary": top1_summary,
        "closed_gap": cg_df,
        "performance_table": perf_table,
    }

print(f"Computed temperature-sampling scoring for {len(temp_tables_by_file)} files.")

# Display per-file tables (stable order)
for p in temp_json_paths:
    fname = p.name
    info = temp_tables_by_file[fname]
    print("\n" + "=" * 80)
    print("File:", fname)
    print("Variant:", info["variant"], "| Temperature:", info["temperature"])
    print("=" * 80)
    print("\nPerformance table:")
    display(info["performance_table"])

# Build an aggregated comparison table across temps/variants
rows = []
for fname, info in temp_tables_by_file.items():
    t = info["temperature"]
    variant = info["variant"]
    perf = info["performance_table"].copy()
    perf["File"] = fname
    perf["Variant"] = variant
    perf["Temperature"] = t
    rows.append(perf)

temp_perf_long = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()
print(f"\nAggregated rows: {len(temp_perf_long)}")
display(temp_perf_long.head(20))

# Pivot view: Single/Parallel score by temperature
if not temp_perf_long.empty:
    single_pivot = temp_perf_long.pivot_table(
        index=["Variant", "Model"],
        columns=["Temperature"],
        values=["Single Score", "Parallel Score", "Closed Gap"],
        aggfunc="first",
    ).sort_index()
    display(single_pivot)

    # Quick summary: best temperature per (Variant, Model) by Single Score
    best_single = (
        temp_perf_long
        .sort_values(["Variant", "Model", "Single Score"], ascending=[True, True, False])
        .groupby(["Variant", "Model"], as_index=False)
        .head(1)
        [["Variant", "Model", "Temperature", "Single Score", "Parallel Score", "Closed Gap"]]
        .rename(columns={"Temperature": "BestTemp"})
    )
    display(best_single)

Loading 10 temperature-sampling JSON files from /home/vro5/Coding/AgenticSolvers/test/data/testOutputSigTemp
Computed temperature-sampling scoring for 10 files.

File: LLMsuggestions_significative_chat_fzn_Sdesc_T0.json
Variant: chat_fzn_Sdesc | Temperature: 0.0

Performance table:


,Model,Single Score,Parallel Score,Closed Gap
0,openai/gpt-oss-120b,49.097559,72.374489,-0.477948



File: LLMsuggestions_significative_chat_fzn_Sdesc_T0p2.json
Variant: chat_fzn_Sdesc | Temperature: 0.2

Performance table:


,Model,Single Score,Parallel Score,Closed Gap
0,openai/gpt-oss-120b,49.07582,73.624489,-0.478875



File: LLMsuggestions_significative_chat_fzn_Sdesc_T0p3.json
Variant: chat_fzn_Sdesc | Temperature: 0.3

Performance table:


...

,Model,Single Score,Parallel Score,Closed Gap
0,openai/gpt-oss-120b,48.581673,74.979358,-0.49995



Aggregated rows: 10


,Model,Single Score,Parallel Score,Closed Gap,File,Variant,Temperature
0,openai/gpt-oss-120b,49.097559,72.374489,-0.477948,LLMsuggestions_significative_chat_fzn_Sdesc_T0...,chat_fzn_Sdesc,0.0
1,openai/gpt-oss-120b,49.075820,73.624489,-0.478875,LLMsuggestions_significative_chat_fzn_Sdesc_T0...,chat_fzn_Sdesc,0.2
2,openai/gpt-oss-120b,50.575820,73.892572,-0.414899,LLMsuggestions_significative_chat_fzn_Sdesc_T0...,chat_fzn_Sdesc,0.3
3,openai/gpt-oss-120b,51.305480,73.374489,-0.383778,LLMsuggestions_significative_chat_fzn_Sdesc_T0...,chat_fzn_Sdesc,0.7
4,openai/gpt-oss-120b,49.825820,73.544802,-0.446887,LLMsuggestions_significative_chat_fzn_Sdesc_T0...,chat_fzn_Sdesc,0.8
5,openai/gpt-oss-120b,50.761038,72.701924,-0.406999,LLMsuggestions_significative_chat_fzn_T0.json,chat_fzn,0.0
6,openai/gpt-oss-120b,52.002097,75.142652,-0.354067,LLMsuggestions_significative_chat_fzn_T0p2.json,chat_fzn,0.2
7,openai/gpt-oss-120b,49.423086,71.379740,-0.464064,LLMsuggestions_significative_chat_fzn_T0p3.json,chat_fzn,0.3
8,openai/gpt-oss-120b,52.843076,73.600511,-0.318199,LLMsuggestions_significative_chat_fzn_T0p7.json,chat_fzn,0.7
9,openai/gpt-oss-120b,48.581673,74.979358,-0.499950,LLMsuggestions_significative_chat_fzn_T0p8.json,chat_fzn,0.8


Closed Gap                                \
Temperature                               0.0       0.2       0.3       0.7   
Variant        Model                                                          
chat_fzn       openai/gpt-oss-120b  -0.406999 -0.354067 -0.464064 -0.318199   
chat_fzn_Sdesc openai/gpt-oss-120b  -0.477948 -0.478875 -0.414899 -0.383778   

                                             Parallel Score             \
Temperature                              0.8            0.0        0.2   
Variant        Model                                                     
chat_fzn       openai/gpt-oss-120b -0.499950      72.701924  75.142652   
chat_fzn_Sdesc openai/gpt-oss-120b -0.446887      72.374489  73.624489   

                                                                     \
Temperature                               0.3        0.7        0.8   
Variant        Model                                                  
chat_fzn       openai/gpt-oss-120b  71.379740  73.600511  74.979358   
chat_fzn_Sdesc openai/gpt-oss-120b  73.892572  73.374489  73.544802   

                                   Single Score                        \
Temperature                                 0.0        0.2        0.3   
Variant        Model                                                    
chat_fzn       openai/gpt-oss-120b    50.761038  52.002097  49.423086   
chat_fzn_Sdesc openai/gpt-oss-120b    49.097559  49.075820  50.575820   

                                                          
Temperature                               0.7        0.8  
Variant        Model                                      
chat_fzn       openai/gpt-oss-120b  52.843076  48.581673  
chat_fzn_Sdesc openai/gpt-oss-120b  51.305480  49.825820

,Variant,Model,BestTemp,Single Score,Parallel Score,Closed Gap
8,chat_fzn,openai/gpt-oss-120b,0.7,52.843076,73.600511,-0.318199
3,chat_fzn_Sdesc,openai/gpt-oss-120b,0.7,51.305480,73.374489,-0.383778
